In [74]:
import pandas as pd
import numpy as np
import os

In [3]:
os.chdir("C:/Users/Cynthia/Desktop/Bootcamp_Github/Basketball-Analysis-Repo/data")

In [6]:
# Draft data

draft = pd.read_csv("orig/1976_to_2015_Draftees.csv")
draft.head(3)

,draft_Player,draft_All_NBA,draft_All.Star,draft_Draft_Yr,draft_Pk,draft_Team,draft_College,draft_Yrs,draft_Games,draft_Minutes.Played,...,draft_Executive,draft_Tenure,draft_Exec_ID,draft_Exec_draft_exp,draft_attend_college,draft_first_year,draft_second_year,draft_third_year,draft_fourth_year,draft_fifth_year
0,Robert Parish,2,9,1976,8,GSW,Centenary College of Louisiana,21,1611,45704,...,Al Attles,3641 days 00:00:00.000000000,1,1,1,0,0,0,0,0
1,Sonny Parker,0,0,1976,17,GSW,Texas A&M University,6,452,10916,...,Al Attles,3641 days 00:00:00.000000000,1,1,1,0,0,0,0,0
2,Marshall Rogers,0,0,1976,34,GSW,University of Texas-Pan American,1,26,176,...,Al Attles,3641 days 00:00:00.000000000,1,1,1,0,0,0,0,0


In [48]:
# NBA season data

nba = pd.read_csv("orig/NBA Season Data.csv")

# Set Roster = 1 if they played the season
nba["Roster"] = 1
nba.head(3)

,nba_Year,nba_Tm,nba_Player,nba_Age,nba_G,nba_MP,nba_PER,nba_TS%,nba_3PAr,nba_FTr,...,nba_Adjusted WORP,nba_Estimated Position,nba_BBRef Pos,nba_Age on Feb 1,nba_Yrs Experience,nba_Height,nba_Weight,nba_Rounded Position,nba_Rounded Age,Roster
0,1978,NOJ,Truck Robinson,26,82,3638,15.7,0.481,NaN,0.340,...,1.1,3.6,4.0,26.33,3.0,79.0,225.0,4.0,26.0,1
1,1978,BUF,Randy Smith,29,82,3314,19.1,0.521,NaN,0.326,...,7.4,1.0,2.0,29.14,6.0,75.0,180.0,1.0,29.0,1
2,1978,WSB,Elvin Hayes,32,81,3246,17.1,0.489,NaN,0.365,...,6.5,4.2,4.0,32.21,9.0,81.0,235.0,4.0,32.0,1


In [55]:
# Match draft data with next year's NBA season data

dfs = []

for y in range(2011,2016):
    draft_yr = draft.loc[draft["draft_Draft_Yr"] == y]
    nba_yr = nba.loc[nba["nba_Year"] == y + 1]
    draft_nba_yr = pd.merge(draft_yr, nba_yr, how = "left",
                            left_on = "draft_Player", right_on = "nba_Player")
    dfs.append(pd.DataFrame(draft_nba_yr))

In [75]:
# Create draft/nba matched data for all years
df = dfs[0]
for i in range(1,5):
    df = df.append(dfs[i])

In [70]:
# Fill Roster w 0 if draft pick did not play the season
df["Roster"] = df["Roster"].fillna(0)

In [ ]:
df.loc[df["Roster"] == 1 & (df["draft_Team"] != df["nba_Tm"])][["draft_Team","nba_Tm"]]

In [79]:
conditions = [
    (df["Roster"] == 1 & (df["draft_Team"] != df["nba_Tm"])),
    (df["Roster"] == 1 & (df["draft_Team"] == df["nba_Tm"]))
]
choices = [1, 0]

df["Traded"] = np.select(conditions, choices)

df["Traded"].value_counts()

0    238
1     72
Name: Traded, dtype: int64

In [49]:
# Save data

df.to_csv("clean/draft_nba.csv", index = False)